# Gestión Moderna de Portafolio
### Autores Bernardo León y Carlos Zapata
### (C) Copyright 2023

## Capitulo 12: Modelo Black-Litterman

In [1]:
#Librerías usadas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%pip install --quiet yfinance
import yfinance as yf
import warnings
warnings.filterwarnings("ignore")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 KB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 KB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 KB 6.4 MB/s eta 0:00:00


In [ ]:
# Información histórica dentro de muetsra (In-sample) para las acciones
fechai = "2015-12-01"
fechaf = "2020-12-31"
periodicidad = "1Mo"
activos = ["ADBE","MCD","MSCI","MSFT","NEE","PG","RSG","WMT"]
precios = yf.download(activos,start=fechai,end=fechaf,interval=periodicidad)['Adj Close'].dropna()
retornos = np.log(precios/precios.shift(1)).dropna()
mu = retornos.mean()*12
cov = retornos.cov()*12
sigma= retornos.std()*np.sqrt(12)
n = len(mu)

[*********************100%***********************]  8 of 8 completed


In [ ]:
indice = ["^GSPC"]
p_indice = yf.download(indice,start=fechai,end=fechaf,interval=periodicidad)['Adj Close'].dropna()
r_indice = np.log(p_indice/p_indice.shift(1)).dropna()
rpm = r_indice.mean()*12
sigmapm = r_indice.std()*np.sqrt(12)

[*********************100%***********************]  1 of 1 completed


In [ ]:
# Estimación de betas 
from scipy import stats
t =len(retornos)
def linestim(x):
  return intercept + slope * x
betas = np.zeros((n,1))

for i in range(n):
  slope, intercept, r, p, se = stats.linregress(r_indice, retornos.iloc[:,i])
  model = list(map(linestim, r_indice))
  betas[i] = slope

betas= pd.DataFrame(betas.T,columns=activos)

In [ ]:
# Retornos de equilibrio
rf = 0
pi = betas * (rpm-rf)

In [ ]:
datos = pd.concat({'Retornos':pd.DataFrame(mu),'Volatilidades':pd.DataFrame(sigma),'Betas':betas.T, 'Pi' : pi.T},axis=1).T
datos

,,ADBE,MCD,MSCI,MSFT,NEE,PG,RSG,WMT
Retornos,0,0.334438,0.145223,0.376101,0.296015,0.243359,0.142153,0.177710,0.194796
Volatilidades,0,0.217030,0.169892,0.214384,0.179247,0.158507,0.141348,0.148402,0.174636
Betas,0,0.956521,0.588163,0.863442,0.803540,0.163976,0.377036,0.672420,0.414432
Pi,0,0.116407,0.071579,0.105080,0.097790,0.019956,0.045885,0.081833,0.050436


In [ ]:
# Pesos portafolio de equilibrio
delta = (rpm - rf)/sigmapm**2
wpequi = ((np.linalg.inv(delta*cov) @ pi.T)/ np.sum(np.linalg.inv(delta*cov) @ pi.T))
wpequi = pd.DataFrame(wpequi.values.T,columns=activos)
wpequi

,ADBE,MCD,MSCI,MSFT,NEE,PG,RSG,WMT
0,0.128907,0.065366,0.147317,0.169343,-0.117659,0.115135,0.401462,0.09013


In [ ]:
# Modelo Black Litterman
tau = 0.025 # Nivel de confiabilidad

# Expectativas: views
q = np.array([[0.035],[0.148],[0.11] ])

P = np.array([[-1, 1, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 1, 0, 0, 0],
              [0, 0, 1, 0, 0, 0, -1, 0]])

omega = np.array([[P[0,:].T@cov@P[0,:],0,0], [0,P[1,:].T@cov@P[1,:],0], [0,0,P[2,:].T@cov@P[2,:]]  ]) #np.matrix(np.diag(np.diag(P.T @ (tau * cov)  P)))

print('Views: ', q)
print('Matriz Link: ', P)
print('Matriz Omega: ', omega)

Views:  [[0.035]
 [0.148]
 [0.11 ]]
Matriz Link:  [[-1  1  0  0  0  0  0  0]
 [ 0  0  0  0  1  0  0  0]
 [ 0  0  1  0  0  0 -1  0]]
Matriz Omega:  [[0.05069737 0.         0.        ]
 [0.         0.02512447 0.        ]
 [0.         0.         0.04542249]]


In [ ]:
def black_litterman(mu, cov, P, q, tau, delta, riskfree):
    P = np.matrix(P)
    PI = np.matrix(pi)
    Q = np.matrix(q)
    S = np.matrix(cov)
    Omega = np.matrix(np.diag(np.diag(P * (tau * S) * P.T))) 
    PI_bl = ((tau * S).I + P.T * Omega.I * P).I * ((tau * S).I * PI.T + P.T * Omega.I * q)
    mu_bl = PI_bl + rf
    return mu_bl.T

In [ ]:
mubl = black_litterman(mu, cov, P, q, tau, delta, rf)
print('Retornos: ', mubl)

Retornos:  [[0.10902828 0.09527464 0.13973944 0.09956609 0.08307738 0.06761536
  0.09116608 0.07601983]]


In [ ]:
wbl = (np.linalg.inv(delta*cov) @ mubl.T)/ np.sum(np.linalg.inv(delta*cov) @ mubl.T)
print('Pesos: ', wbl.T)

Pesos:  [[-0.02942701  0.1623099   0.26614044  0.11583138  0.23551777  0.07875241
   0.10922624  0.06164885]]
